In [1]:
from dataclasses import dataclass
from kv import KV, ClientKV

In [13]:
kv = KV.of('http://localhost:8000;Token=secret', dict)

In [14]:
await kv.insert('hello2', dict(hello='world'))

Left(value=InvalidData(detail={}, reason='invalid-data'), tag='left')

In [9]:
await kv.values().sync()

[Right(value={'hello': 'world'}, tag='right'),
 Right(value={'hello': 'world'}, tag='right')]

In [5]:
for user in users:
  (await kv.insert(user.uuid, user)).unsafe()

In [6]:
await kv.clear()

Right(value=None, tag='right')